In [1]:
import sqlite3
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain_ollama.llms import OllamaLLM
from langchain.chains import create_sql_query_chain
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools import QuerySQLDataBaseTool
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

In [31]:
conn = sqlite3.connect('pazar3.db')
cursor = conn.cursor()

# Create table with the selected schema
cursor.execute('''
CREATE TABLE pazar3(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Title TEXT,
    Price FLOAT,
    Currency TEXT,
    Number_of_rooms FLOAT,
    Location TEXT,
    'Area(m2)' FLOAT
)
''')

df = pd.read_csv('pazar3_properties.csv')

df.to_sql('pazar3', conn, if_exists='append', index=False)

cursor.close()
conn.close()

In [33]:
db = SQLDatabase.from_uri("sqlite:///pazar3.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM pazar3 LIMIT 1;")

sqlite
['pazar3']


"[(1, 'Gradezna parcela', None, None, None, 'Vasilevo, Strumica', 3000.0)]"

In [34]:
llm = OllamaLLM(model="llama3.1")

In [59]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "What property has the biggest Area(m2)?"})

In [60]:
x = response.split(":")[-1].strip()

In [61]:
x

'SELECT "Title", "Area(m2)" FROM pazar3 ORDER BY "Area(m2)" DESC LIMIT 1'

In [62]:
db.run(x)

"[('Pkliste 14 ektara Kriva palanka', 140000.0)]"

In [48]:
answer_prompt = PromptTemplate.from_template(
    """Based on the user's question, the corresponding SQL query, and the SQL result, answer the user's question.

Question: {question}
SQLQuery: {query}
SQLResult: {result}
Answer: """
)

In [49]:
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)

In [70]:
# Specify the database to connect to
db = SQLDatabase.from_uri("sqlite:///pazar3.db")

# LLM settings
llm = OllamaLLM(model="llama3.1")

# Tool to execute SQL statements
execute_query = QuerySQLDataBaseTool(db=db)

# Chain to generate SQL statements
write_query = create_sql_query_chain(llm, db)

# Prompt template to generate answers in natural language
answer_prompt = PromptTemplate.from_template(
    """Based on the user's question, the corresponding SQL query, and the SQL result, answer the user's question.

Question: {question}
SQLQuery: {query}
SQLResult: {result}
Answer: """
)

# Chain to generate answers using the above prompt
answer = answer_prompt | llm | StrOutputParser()

# Chain to pass the SQL query result to the answer chain
chain = (
    RunnablePassthrough.assign(
        query=write_query
    ).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

# Execute the query
answer = chain.invoke({"question": "What property has the biggest Area(m2)?"})

In [71]:
print(answer)

It looks like there's a bit of confusion here!

The issue is that the SQL query and result you provided are not related to the question. The "Error" message at the top suggests that SQLite is expecting a valid SQL query, but it's being given a string that starts with "Question: What property has the biggest Area(m2)?", which is not a valid SQL query.

Let's try again!

Assuming you want to know which property has the largest area in square meters, I'll provide an answer based on the correct SQL query:

The SQL query `SELECT "Title", "Area(m2)" FROM pazar3 ORDER BY "Area(m2)" DESC LIMIT 1` is trying to find the row with the highest "Area(m2)" value and return its corresponding "Title".

If we assume this query runs without errors, the answer would be: The property that has the biggest Area(m2) is... (wait for it)... **the one with the maximum value of "Area(m2)"**. Unfortunately, since we don't have the actual result set, I can't tell you which specific property that is.


In [65]:
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "What property has the biggest Area(m2)??"})

'Error: (sqlite3.OperationalError) near "Question": syntax error\n[SQL: Question: What property has the biggest Area(m2)?\nSQLQuery: \nSELECT DISTINCT "Title", "Area(m2)" FROM pazar3 ORDER BY "Area(m2)" DESC LIMIT 1]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [89]:
from langchain_community.agent_toolkits import create_sql_agent

db = SQLDatabase.from_uri("sqlite:///pazar3.db")

# LLM settings
llm = OllamaLLM(model="llama3.1")

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm = llm,
    toolkit = toolkit,
    verbose = True,
    handle_parsing_errors=True
)

In [90]:
agent_executor.run("What is the property with the biggest Area(m2)?")



> Entering new SQL Agent Executor chain...
Thought: I should list all the tables in the database and then check their schema.
Action: sql_db_list_tables
Action Input:pazar3Action: sql_db_schema
Action Input: pazar3
CREATE TABLE pazar3 (
	id INTEGER, 
	"Title" TEXT, 
	"Price" FLOAT, 
	"Currency" TEXT, 
	"Number_of_rooms" FLOAT, 
	"Location" TEXT, 
	"Area(m2)" FLOAT, 
	PRIMARY KEY (id)
)

/*
3 rows from pazar3 table:
id	Title	Price	Currency	Number_of_rooms	Location	Area(m2)
1	Gradezna parcela	None	None	None	Vasilevo, Strumica	3000.0
2	Се издава нов стан 76 m2 до ОУ Блаже Конески	550.0	EUR	4.0	Aerodrom, Skopje	76.0
3	SHITEN SHTEPIT DHE PARCELAT	1.0	EUR	None	Kumanovo	1600.0
*/Question: What is the property with the biggest Area(m2)?
Thought: I should now query the database to get the property with the largest area.
Action: sql_db_query_checker
Action Input: SELECT "Area(m2)" FROM pazar3 ORDER BY "Area(m2)" DESC LIMIT 10SELECT "Area(m2)" FROM pazar3 ORDER BY "Area(m2)" DESC LIMIT 10Questi

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Question: What is the property with the biggest Area(m2)?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Thought: I should list all the tables in the database and then check their schema.
Action: sql_db_list_tables
Action Input`